We’re going to make some simple simulated neural data and play with analyzing it. Our super-simple “neuron” will have almost no dynamics in it, and will not be very realistic at all, but it will be useful nevertheless. Later in the class we’ll deal with some more realistic simulations and data.

We’re going to simulate an experiment in which you’ve patched onto a neuron in a slice (the neuroscientists can explain what this means), stimulate it with a current, and record the resulting membrane potential. Then we’ll identify moments when the simulated neuron would spike, and we’ll look at those spike times. We’ll bin them in time, to get estimates of number of spikes/sec for each time bin. And then we’ll look at what happens when we stimulate two independent neurons. 

The final questions we’ll asses are, “how correlated are the firing rates of the two neurons, and how does the strength of that correlation depend on experimental parameters such as number of trials and stimulation amplitude?”

Since this problem set will make use of NumPy and matplotlib, we'll import them right now.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

<hr>
## Problem 1

Make a vector, called ```t```, that will stand for your time axis. Its elements should climb, in uniform steps of 0.001 (i.e., one millisecond) from 0 to 2 seconds.

In [ ]:
# define vector t
#<answer>
t = np.arange(0, 2, 0.001)
#</answer>

Now make a vector called ```drive``` that is the same length as t and whose elements are a sine wave, frequency 4 cycles/sec, with a mean of zero. The amplitude of the sine wave modulation should be ```drive_amplitude``` (this is a parameter that we will later change, so it is good to define it as a variable, not as a fixed number). 

In [ ]:
# define drive vector
#<answer>
drive_amplitude = 5.0
drive_frequency = 4 # Hz
drive = drive_amplitude * np.sin(2*np.pi*drive_frequency * t)
#</answer>

Verify your work: when ```drive_amplitude=1```, if you ```plt.plot(t, drive)```, you should see an amplitude 1 sine wave, of 8 cycles total (4 in each of the two seconds).

In [ ]:
# plot the drive vector
#<answer>
plt.plot(t, drive)
plt.xlabel('Time (sec)')
plt.ylabel('Amplitude')
#</answer>

<hr>
## Problem 2

Now we’ll simulate injecting this drive (think of it as a current) into a neuron. 

Write a function called ```neuron``` that takes a drive vector as its input parameter. This function should create a simulated membrane potential that, at each timestep, is -60 (the resting membrane potential, in millivolts) plus a sample from a gaussian distribution with standard deviation 5 (to represent the typical neural noise observed in experiments; the ```np.random.normal``` function will be useful here) plus the value of the drive at that timestep. In other words, the function produces $v$:

<center> $v(t) = -60 + N(0,5) + d(t)$ </center>

where $d(t)$ is the drive signal supplied to the function.

If the voltage goes above -50, we’ll say it has reached threshold and the neuron spikes: replace each element of your membrane potential vector that is greater than -50 with +10, the peak voltage of a spike. (Our super-simple neuron has no dynamics in that the membrane potential at one timestep doesn’t depend on any other timesteps; in week 3 we’ll change that and go to more realistic neurons.) 

Your ```neuron``` function should return a vector that represents the final membrane potential, ```v```.  Plot ```v``` versus ```t``` to see what ```v``` looks like. Run it several times; since you’re using random numbers, it should look different each time.

In [ ]:
# define the neuron function
#<answer>
def neuron(drive):
    v = -60 + np.random.normal(0, 5, size=drive.shape) + drive
    v[v>-50] = 10
    return v
#</answer>

In [ ]:
# plot the result of using the neuron function
#<answer>
plt.plot(t, neuron(drive))
plt.xlabel('Time (sec)')
plt.ylabel('Membrane Potential (mV)')
#</answer>

<hr>
## Problem 3

We will call each time you run your neuron a “trial.” Run the neuron for 100 trials with ```drive_amplitude=1```, and plot the resulting spike times in a “raster plot:” this is a plot in which each row of the y-axis represents one trial, and the x-axis represents time. At the time of each spike in each trial, you should place a small vertical line in your plot to indicate that a spike occurred.

In [ ]:
# generate and display a raster plot
#<answer>
drive_amplitude = 5
drive = drive_amplitude * np.sin(2*np.pi*drive_frequency * t)

for i in range(100):
    v = neuron(drive)
    spike_times = t[v>-50]
    plt.vlines(spike_times, i, i+1)
    
plt.xlabel('Time (sec)')
plt.ylabel('Trial #')
#</answer>

What does the raster plot look like for different numbers of trials and different drive amplitudes? Can you see the sine pattern? (You can edit that cell and run it again.)

<hr>
## Problem 4

We will call a set of n trials an “experiment.” Divide the time in each trial into bins of b milliseconds (start with b=25 milliseconds). 

For each trial, and for each of these time bins, find the number of spikes fired by the neuron. Divide by b to obtain, in each bin and for each trial, an estimate of the number of spikes/sec fired by the neuron. This is an estimate of its firing rate. 

Using ```plt.imshow```, plot the estimated firing rates for each trial as a function of time during each trial.

In [ ]:
# generate and display a firing rate map
plt.figure('Answer to Problem 4') # we suggest you create a figure for each answer. Here is one for Problem 4
#<answer>
n_trials = 100

# setup bins of width b to bin spikes
b = 0.025
bins = np.arange(min(t), max(t)+b, b)
firing_rates = np.zeros([n_trials, len(bins)-1])

for trial in range(n_trials):
    v = neuron(drive)
    spike_times = t[v>-50]
    n_spikes,_ = np.histogram(spike_times, bins) # binned spike counts
    firing_rates[trial] = n_spikes / b
    
plt.imshow(firing_rates, interpolation='none', cmap=plt.cm.viridis)

_ = plt.xticks(range(len(bins))[::8], bins[::8], rotation=90) # set the x-axis to display time
plt.colorbar(label='Firing Rate (Hz)')
plt.xlabel('Time (sec)')
plt.ylabel('Trial #')
#</answer>

<hr>
## Problem 5

Take your estimated firing rates, and average over trials to obtain an estimated firing rate as a function of time that we’ll call the “PeriStimulus Time Histogram” or PSTH. Plot the PSTH. What does the PSTH look like for different numbers of trials and different drive amplitudes?

In [ ]:
# generate and display a PSTH
plt.figure('Answer to Problem 5') # we suggest you create a figure for each answer. Here is one for Problem 5

#<answer>
psth = firing_rates.mean(axis=0)

bin_centers = (bins[1:] + bins[:-1]) / 2
plt.bar(bin_centers, psth, width=b)
plt.xlabel('Time (sec)')
plt.ylabel('Firing Rate (Hz)')
#</answer>

<hr>
## Problem 6

Now run *two* experiments, both with same number of trials and the same drive amplitude, to represent two neurons that you patched simultaneously, injected current into simultaneously, and recorded simultaneously. Compute the two PSTHs that correspond to these two neurons in the two experiments. 

In [ ]:
# compute the PSTHs for two simultaneously patched neurons
#<answer>
# parameters for this experiment
n_trials = 200
drive_amplitude = 10
b = 0.025

drive = drive_amplitude * np.sin(2*np.pi*drive_frequency * t)
bins = np.arange(min(t), max(t)+b, b)
firing_rates = np.zeros([2, n_trials, len(bins)-1]) # 2 represents the 2 neurons

for trial in range(n_trials):
    v1 = neuron(drive) # neuron 1
    v2 = neuron(drive) # neuron 2
    spike_times_1 = t[v1>-50]
    spike_times_2 = t[v2>-50]
    n_spikes_1,_ = np.histogram(spike_times_1, bins) # binned spike counts
    n_spikes_2,_ = np.histogram(spike_times_2, bins) # binned spike counts
    firing_rates[0, trial] = n_spikes_1 / b
    firing_rates[1, trial] = n_spikes_2 / b
    
psth = firing_rates.mean(axis=1)
#</answer>

Calculate the correlation coefficient between the two PSTHs. (If $\sigma_x$ and $\sigma_y$ are the standard deviation of $x$ and $y$, respectively, and $\langle \rangle$ represents averaging, the correlation coefficient is defined as:

<center> $\frac{\langle xy \rangle - \langle x \rangle \langle y \rangle}{\sigma_x \sigma_y}$ </center>

In [ ]:
# compute the correlation coefficient of the two PSTHs
#<answer>
def corrcoef(x,y):
    """Calcualte the correlation coefficient of two signals *x* and *y*
    Note that this can be accomplished using np.corrcoef, but this function is here for demonstration.
    """
    return ( (x*y).mean() - x.mean()*y.mean() ) / ( x.std() * y.std() )

print(corrcoef(*psth))
#</answer>

<hr>
## Problem 7

Run the pair of experiments (as in problem #6) many times, using different numbers of trials, and different drive amplitudes. 

To do this, it may be useful to first convert your solution to problem #6 into a function.

In [ ]:
# write a function that performs your solution to problem 6, returning the correlation coefficient
#<answer>
def paired_patch(n_trials=200, drive_amplitude=10, b=0.010):
    """Performs a simulated experiment where 2 neurons are simultaneously patched
    
    Parameters
    ----------
    n_trials : int
        number of trials in the experiments
    drive_amplitude : float
        amplitude of drive signal
    b : float
        bin size for psth generation
        
    Returns
    -------
    Correlation coefficient between the two cells' signals
    """

    drive = drive_amplitude * np.sin(2*np.pi*drive_frequency * t)
    bins = np.arange(min(t), max(t)+b, b)
    firing_rates = np.zeros([2, n_trials, len(bins)-1]) # 2 represents the 2 neurons

    for trial in range(n_trials):
        v1 = neuron(drive) # neuron 1
        v2 = neuron(drive) # neuron 2
        spike_times_1 = t[v1>-50]
        spike_times_2 = t[v2>-50]
        n_spikes_1,_ = np.histogram(spike_times_1, bins) # binned spike counts
        n_spikes_2,_ = np.histogram(spike_times_2, bins) # binned spike counts
        firing_rates[0, trial] = n_spikes_1 / b
        firing_rates[1, trial] = n_spikes_2 / b

    psth = firing_rates.mean(axis=1)

    return np.corrcoef(*psth)[0,1]
#</answer>

Use plt.imshow to plot the correlation coefficient as a function of drive amplitude and number of trials. 

In [ ]:
# use your function to generate a map of correlation coefficients for ranges of parameter values
plt.figure('Answer to Problem 7') # we suggest you create a figure for each answer. Here is one for Problem 7
#<answer>
n_trials_list = np.arange(2, 30, 1)
drive_amplitudes_list = np.linspace(0.5, 10, 40)

cc = np.zeros([len(n_trials_list), len(drive_amplitudes_list)])

for t_idx,n_trials in enumerate(n_trials_list):
    for d_idx,drive_amplitude in enumerate(drive_amplitudes_list):
        cc[t_idx, d_idx] = paired_patch(n_trials=n_trials, drive_amplitude=drive_amplitude)
        
plt.imshow(cc, origin='lower', interpolation='none', cmap=plt.cm.inferno)
_ = plt.xticks(range(len(drive_amplitudes_list))[::5], drive_amplitudes_list[::5].astype(int))
_ = plt.yticks(range(len(n_trials_list))[::5], n_trials_list[::5])
plt.xlabel('Drive Amplitude')
plt.ylabel('# Trials')
plt.colorbar(label='Correlation Coefficient')
#</answer>

Replot the same data as a “surface plot” using matplotlib. If you’re trying to observe a correlation coefficient, you could use this simulation and its final plot to try to decide what drive amplitude to use, and how many trials to record from, in each experiment. Does correlation coefficient become stronger or weaker as number of trials or drive amplitude grow?

To get you started, some sample code is included.
* ```drive_amplitudes_list```: the list of drive amplitudes you tested
* ```n_trials_list```: the list of n_trials you tested
* ```cc```: the list of resulting correlation coefficients

In [ ]:
# fill in these 3 lists with your work
drive_amplitudes_list = [1,2,3] # change this to your own values
n_trials_list = [2,3,4] # change this to your own values
cc = = [0.5, 0.5, 0.5] # change this to your own values

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')

X,Y = np.meshgrid(drive_amplitudes_list, n_trials_list)

ax.plot_surface(X, Y, cc)
ax.set_xlabel('Drive Amplitude')
ax.set_ylabel('# Trials')
ax.set_zlabel('Correlation Coefficient')

plt.tight_layout()